# **MATPLOTLIB** и немного музыки

посмотрим на тестовый датасет с данными о прослушиваниях музыки с **last.fm** (до 2018 года видимо) и поработаем с ним

то, что снизу, не особо важно, подгружаю данные, чтоб было что рисовать

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

data = pd.read_csv('/kaggle/input/music-artists-popularity/artists.csv', low_memory=False);

In [ ]:
def prepare_data(data):
    #simple preprocessing
    data = data.sample(frac=1) #shuffle (not to see the most popular artist)
    data = data[['artist_lastfm', 'country_lastfm', 'tags_lastfm', 'listeners_lastfm', 'scrobbles_lastfm', 'ambiguous_artist']] #interesting columns
    data = data.dropna(subset=['artist_lastfm', 'listeners_lastfm', 'scrobbles_lastfm']) #remove NaN data
    data = data.query('listeners_lastfm>100') #remove full underground
    data = data.query('ambiguous_artist==False') #remove dirty data
    data = data.drop_duplicates(subset=['artist_lastfm', 'listeners_lastfm', 'scrobbles_lastfm', 'ambiguous_artist']) #remove duplicates
    data.drop('ambiguous_artist', axis=1, inplace=True)
    data['artist_lastfm'] = data.artist_lastfm.replace(r'[^\w\d\s]', '', regex=True) #simplify artist names
    return data.reset_index(drop=True)

In [ ]:
df = prepare_data(data)

In [ ]:
df.head(10) #наши данные

Вернёмся к теме. Сначала посмотрим на игрушечные примеры, потом изучим реальные данные

## matplotlib

Мощный инструмент для визуализации графиков, диаграмм и т.п.

In [ ]:
import matplotlib.pyplot as plt

# для jupyter notebooks
%matplotlib inline

## [**plt.plot**](https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.plot.html) - график зависимости **y** от **x** 

In [ ]:
# нарисовать синус и косинус
x = np.linspace(0, 2.*np.pi, 100) # создать массив из 100 элементов, равномерно распределенных внутри интервала 0, 2*np.pi
plt.plot(x, np.sin(x)) # нарисовать график синуса (сначала массив элементов по оси x, затем по y)
plt.plot(x, np.cos(x))

plt.show() # в jupyter необязательно прописывать
plt.savefig('fig1.png') # сохранить картинку в png-файл

In [ ]:
#теперь сделаем красиво

x = np.linspace(0, 1, 60)
f1 = 0.25 - (x - 0.5)**2
f2 = x**3
plt.plot(x, f1, ':b')    # пунктирная синяя линия
plt.plot(x, f2, '--r')   # штрихованная красная линия
plt.plot(x, f1+f2, 'k')  # черная непрерывная линия

plt.show() # в jupyter необязательно прописывать

In [ ]:
y = np.random.random(10)

plt.plot(y, 's'); # если передать только один список, то он будет использован для значений по вертикальной оси

## [**plt.errorbar**](https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.errorbar.html?highlight=errorbar#matplotlib.pyplot.errorbar) = **plt.plot** + ошибки измерений

In [ ]:
x = np.arange(6)
y = np.random.poisson(15, x.size) # например считаем, сколько снежинок попало нам в руку за каждую минуту наблюдения
y_error = np.sqrt(y) # для пуассоновского распредения ошибка равна корню из измерения 

plt.errorbar(x, y, yerr=np.sqrt(y), marker='o', linestyle='none') # yerr - список ошибок измерений 

plt.show()

также можно настроить отображение ошибок по оси **x**, асимметричных ошибок, их вид и т.п.

Обо всём этом можно прочитать на [nsu-programming](https://nsu-programming.github.io/textbook/python/plotting) и послушать на следующей лекции

## [**plt.hist**](https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.hist.html?highlight=hist#matplotlib.pyplot.hist) - гистограммы

In [ ]:
data = np.random.poisson(145, 10000) # тысяча измерений данных, распределённых согласно пуассоновскому закону со средним 145
plt.hist(data, bins=200); # рисуем гистограмму на 200 бинов

In [ ]:
plt.hist(data, bins=40, range=(80, 220)); # range=(float, float) диапазон значений, в котором строится гистограмма

In [ ]:
plt.hist(data, bins=40, range=(80, 220), histtype='step'); # стиль гистограммы histtype может быть {'bar', 'barstacked', 'step', 'stepfilled'}

также в `matplotlib` можно настроить огромное количество параметров, связанных с гистограммами (определить веса бинов, нормировать гистограмму, строить сумму гистограмм и т.д.)

И об этом тоже можно прочитать на [nsu-programming](https://nsu-programming.github.io/textbook/python/plotting) и послушать следующую лекцию

## [**plt.scatter**](https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.scatter.html) - диаграмма рассеяния

In [ ]:
means = (0.5, 0.9)
covar = [
    [1., 0.6],
    [0.6, 1.]
]
data = np.random.multivariate_normal(means, covar, 5000) # намоделировали 5000 точек двумерного нормального распределения

data_add = np.random.multivariate_normal((-1, -0.2), [[1, -0.2], [-0.2, 1]], 5000) # намоделировали ещё 5000 точек двумерного нормального распределения

In [ ]:
plt.scatter(data[:, 0], data[:, 1]);

цели своей мы добились, отрисовали, но довольно некрасиво

## делаем красиво

In [ ]:
# установить разрешение картинки (чем выше dpi, тем выше качество)
plt.figure(dpi=120, figsize=(5, 5))

plt.scatter(data[:, 0], data[:, 1], s=1, label='My Data')
# s - размер точки на диаграмме рассеяния
# label - название данных

# добавим другие данные на диаграмму
plt.scatter(data_add[:, 0], data_add[:, 1], s=1, label='Add Data', alpha=0.5, marker='v')
# alpha - коэффициент прозрачности (1 - непрозрачные точки, 0 - невидимые)
# marker - тип точек на диаграмме рассеяния (здесь, 'v' - треугольники)

# включить крупную сетку
plt.grid(ls='--') 

# # включить мелкую сетку
plt.grid(which='minor', ls=':')
plt.minorticks_on()

# установить диапазон отрисовки по осям х и y
plt.xlim(-5, 5) 
plt.ylim(-5, 5)

# добавить подписи графика, осей x и y
plt.title('Title')
plt.xlabel('x-axis')
plt.ylabel('y-axis')

# добавить легенду
plt.legend();

теперь попробуем нарисовать красивые гистограммы

In [ ]:
hist_dat1 = np.random.normal(4, 4, 10000)
hist_dat2 = np.random.normal(-5, 5, 20000)

In [ ]:
#отрисовка по дефолту
plt.hist(hist_dat1)
plt.hist(hist_dat2);

In [ ]:
# установить разрешение картинки
plt.figure(dpi=120)

plt.hist(hist_dat1, bins=100, range=(-30, 30), alpha=0.6, label='Data1', density=True, color='purple');
plt.hist(hist_dat2, bins=100, range=(-30, 30), alpha=0.6, label='Data2', density=True, color='seagreen');
# bins - количество бинов
# range - диапазон, в котором будет рассматриваться гистограмма
# alpha - задать прозрачность
# label - определить название данных
# density - если True, то нормировать гистограмму на 1
# color - цвет гистограммы

# включить сетки
plt.grid(ls='--', alpha=0.4)
plt.grid(which='minor', ls=':', alpha=0.3)
plt.minorticks_on()

# добавить подписи графика, осей x и y
plt.title('Distributions histograms')
plt.xlabel('x axis')
plt.ylabel('probability density')

# добавить легенду
plt.legend();

In [ ]:
#Отрисовать те же две гистограммы, но каждую в своём окне

# подготовка окна
fig = plt.figure(dpi=120, figsize=(12,4)) # figsize = (width, height) - размер окна 

ax1, ax2 = fig.subplots(1, 2) # fig.subplots(n_rows, n_cols) - получаем оси, n_rows/n_cols - количество строк/столбцов на картинке

# рисуем в осях ax1
ax1.hist(hist_dat1, bins=100, range=(-30, 30), alpha=0.6, label='Data1', density=True, color='purple');
# рисуем в осях ax2
ax2.hist(hist_dat2, bins=100, range=(-30, 30), alpha=0.6, label='Data2', density=True, color='seagreen');

# теперь декор нужно проделывать для каждой из осей
for ax in (ax1, ax2):
    # включить сетки
    ax.grid(ls='--', alpha=0.4)
    ax.grid(which='minor', ls=':', alpha=0.3)
    ax.minorticks_on()

    # добавить подписи графика, осей x и y
    ax.set_title('Distributions histograms')
    ax.set_xlabel('x axis')
    ax.set_ylabel('probability density')

    # добавить легенду
    ax.legend();

## Теперь музыка

In [ ]:
df.head()
# artist_lastfm - имя артиста
# country_lastfm - страна
# tags_lastfm - тэги
# listeners_lastfm - количество человек, слушающих исполнителя
# scrobbles_lastfm - количество прослушиваний данного исполнителя

## (1) Гистограмма по прослушиваниям

In [ ]:
plt.figure(dpi=120)

plt.hist(df.listeners_lastfm, bins=100, color='firebrick', alpha=0.7);

# включить сетки
plt.grid(ls='--', alpha=0.4)
plt.grid(which='minor', ls=':', alpha=0.3)
plt.minorticks_on()

plt.title('Распределение исполнителей\nпо уникальным слушателям')
plt.xlabel('Количество уникальных слушателей')
plt.ylabel('Количество артистов')

plt.xlim(0, None)

# масштаб по оси y
plt.yscale('log'); # может быть {'linear', 'log', 'symlog'}

In [ ]:
plt.figure(dpi=120)

plt.hist(df.scrobbles_lastfm, bins=100, color='peru');

# включить сетки
plt.grid(ls='--', alpha=0.4)
plt.grid(which='minor', ls=':', alpha=0.3)
plt.minorticks_on()

plt.title('Распределение исполнителей\nпо прослушиваниям')
plt.xlabel('Количество прослушиваний')
plt.ylabel('Количество артистов')

plt.xlim(0, None)

# масштаб по оси y
plt.yscale('log'); # может быть {'linear', 'log', 'symlog'}

* заметим, как много музыкантов в первом бине (и это я ещё отфильтровал датасет)
* теперь предположения, кто в топ-10 по прослушиваниям и уникальным слушателям

## (2) **scatterplot** для первых топ-20

In [ ]:
top20_list = df.sort_values(by='listeners_lastfm', ascending=False).iloc[:20]
top20_scr  = df.sort_values(by='scrobbles_lastfm', ascending=False).iloc[:20]
top = top20_list.append(top20_scr).drop_duplicates()

In [ ]:
# добавляю условный массив размеров для точек на след. картинке, чтоб продемонстрировать возможности matplotlib
sizes = (top.listeners_lastfm/top.listeners_lastfm.max())**2 + (top.scrobbles_lastfm/top.scrobbles_lastfm.max())**2
top['sizes'] = 20 + 80*(sizes - sizes.min())/(sizes.max() - sizes.min());

In [ ]:
plt.figure(dpi=120)

plt.scatter(top.listeners_lastfm, top.scrobbles_lastfm, marker='*', c='darkred', s=top.sizes);
# s - размер точек (здесь берётся из массива sizes)

# включить сетки
plt.grid(ls='--', alpha=0.4)
plt.grid(which='minor', ls=':', alpha=0.3)
plt.minorticks_on()

plt.title('Топ-исполнители')
plt.ylabel('Количество прослушиваний')
plt.xlabel('Количество слушателей')

# добавляю подписи к пяти наибольшим точкам
for _, row in top.sort_values(by='sizes', ascending=False).head(5).iterrows():
    plt.annotate(row.artist_lastfm, (row.listeners_lastfm, row.scrobbles_lastfm), 
                 textcoords="offset points",
                 xytext=(-1,2.5),
                 horizontalalignment='right')

In [ ]:
top

отлично, самых популярных исполнителей нашли и посмотрели, теперь новая задача

## (3) топ-20 для России

In [ ]:
df_ru = df.loc[df.country_lastfm.fillna('').str.endswith('Russia')].copy()

top20_ru_list = df_ru.sort_values(by='listeners_lastfm', ascending=False).iloc[20::-1].reset_index(drop=True).replace('$', '')
top20_ru_scr  = df_ru.sort_values(by='scrobbles_lastfm', ascending=False).iloc[20::-1].reset_index(drop=True)
top_ru = top20_ru_list.append(top20_ru_scr).drop_duplicates().reset_index(drop=True)

данные не всегда корректны, например last.fm не различает разных исполнителей YG между собой, поэтому явно не тот YG войдёт в следующий топ

In [ ]:
fig, ax = plt.subplots(figsize=(8,5), dpi=100)

xmin, xmax = top20_ru_list.listeners_lastfm.min()*0.95, top20_ru_list.listeners_lastfm.max()*1.05

ax.hlines(y=top20_ru_list.index, xmin=xmin, xmax=xmax, 
          color='gray', alpha=0.7, linewidth=1, linestyles='dashdot')
ax.scatter(y=top20_ru_list.index, x=top20_ru_list.listeners_lastfm, s=80, color='crimson', alpha=0.7)
ax.set_yticks(top20_ru_list.index)
ax.set_yticklabels(top20_ru_list.artist_lastfm, fontdict={'horizontalalignment': 'right'});

ax.set_title('Топ российских артистов по уникальным слушателям')
ax.set_xlabel('Количество уникальных слушателей');

ax.set_xlim(xmin, xmax);

In [ ]:
fig, ax = plt.subplots(figsize=(8,5), dpi=100)

xmin, xmax = top20_ru_scr.scrobbles_lastfm.min()*0.95, top20_ru_scr.scrobbles_lastfm.max()*1.05

ax.hlines(y=top20_ru_scr.index, xmin=xmin, xmax=xmax, 
          color='gray', alpha=0.7, linewidth=1, linestyles='dashdot')
ax.scatter(y=top20_ru_scr.index, x=top20_ru_scr.scrobbles_lastfm, s=80, color='crimson', alpha=0.7)
ax.set_yticks(top20_ru_scr.index)
ax.set_yticklabels(top20_ru_scr.artist_lastfm, fontdict={'horizontalalignment': 'right'});

ax.set_title('Топ российских артистов по прослушиваниям')
ax.set_xlabel('Количество прослушиваний');

ax.set_xlim(xmin, xmax);

## (4) Среднее количество прослушиваний исполнителя одним пользователем

In [ ]:
top10 = top.sort_values(by='sizes', ascending=False).iloc[:10]
top10['scr_per_lis'] = top10.scrobbles_lastfm/top10.listeners_lastfm

In [ ]:
plt.figure(dpi=120)

plt.plot(range(len(top10.scr_per_lis)), top10.scr_per_lis, '--ko', ms=7)

# включить сетки
plt.grid(ls='--', alpha=0.4)
plt.grid(which='minor', ls=':', alpha=0.2)
plt.minorticks_on()

plt.ylim(0, None)

plt.xticks(range(len(top10.scr_per_lis)), top10.artist_lastfm, rotation=90);
plt.ylabel('Среднее количество прослушиваний');

plt.tight_layout() # функция позволяет избежать ситуации, когда часть изображения обрежется при сохранении
plt.savefig('рисунок1.png') # сохранить картинку

## (5) распространённость тегов по странам

In [ ]:
def tags_by_country(country):
    return df.loc[df.country_lastfm.fillna('').str.endswith(country)].tags_lastfm.str.lower()\
                .str.split(';').explode().str.replace(r'[^\w\d]', '', regex=True).str.strip().values

ru_tags = tags_by_country('Russia')
uk_tags = tags_by_country('United Kingdom')
us_tags = tags_by_country('United States')
ge_tags = tags_by_country('Germany')

In [ ]:
def get_count_tags(tags_list, tags_use):
    tags_list = tags_list[np.isin(tags_list, tags_use)]
    tag, count = np.unique(tags_list, return_counts=True)
    return pd.Series(count, index=tag)

In [ ]:
genres = ['rock', 'pop', 'hiphop', 'alternative', 'electronic', 'indie']

In [ ]:
df_tags = pd.DataFrame({'US': get_count_tags(us_tags, genres),
                        'UK': get_count_tags(uk_tags, genres),
                        'RU': get_count_tags(ru_tags, genres),
                        'GE': get_count_tags(ge_tags, genres),
                       })
df_tags[df_tags.columns] = df_tags.values.argsort(axis=0).argsort(axis=0)
df_tags = df_tags.T

In [ ]:
from scipy.interpolate import make_interp_spline, BSpline
import matplotlib.cm as cm

In [ ]:
cmap = cm.get_cmap('ocean')

plt.figure(dpi=120, figsize=(10, 4))

x = np.arange(len(df_tags))
x_smo = np.linspace(x.min(), x.max(), 200) 
for i, col in enumerate(df_tags.columns[::-1]):
    y = df_tags[col]
    spl = make_interp_spline(x, y, k=2)
    y_smo = spl(x_smo)
    m_smo = np.linspace(0, x_smo.size-1, len(x)).astype(int)
    plt.plot(x_smo, y_smo, '-', label=col, lw=3, marker='o', ms=13, c=cmap(i/len(df_tags.columns)), alpha=0.6, markevery=m_smo.tolist())
    
plt.title('Относительная популярность жанров\nпо странам среди представленных')
plt.grid(ls='--')
plt.legend()
plt.yticks(np.arange(len(df_tags.columns)), [f'{i+1} place' for i in range(len(df_tags.columns))][::-1]);
plt.xticks(x, df_tags.index);